In [1]:
"""
========================================================
Author:  Sevan Harootonian
Affiliation: Princeton University
Date: 2025-08-20
========================================================

Description:
------------
This notebook includes the following computations for experiment 1
  - Model simulations to compute utility (about 30min) [will use about 10gb of ram]
  - Model fitting to the data (about 5 min)
  - Model simulation using the fitted parameters (100 iteration ~20min)
  - Model fitting for model recovery (100 iteration ~ 14 hours)
  
========================================================
"""

'\n========================================================\nAuthor:  Sevan Harootonian\nAffiliation: Princeton University\nDate: 2025-08-20\n========================================================\n\nDescription:\n------------\nThis notebook includes the following computations for experiment 1\n  - Model simulations to compute utility (about 30min) [will use about 10gb of ram]\n  - Model fitting to the data (about 5 min)\n  - Model simulation using the fitted parameters (100 iteration ~20min)\n  - Model fitting for model recovery (100 iteration ~ 14 hours)\n  \n========================================================\n'

In [ ]:
# model simulation for exp 1 (this wil take about 30min) about 10gb of ram

import pandas as pd
from functions.mdp_params import make_true_graph,create_true_mdp_params
from functions.mentor import OptimalBayesianMentor, NaiveBayesianMentor, PriorOnlyMentor
from functions.max_utility_models import q_values, pathAvgUtility
from msdm.algorithms import ValueIteration
from functions.functions import create_mdp
from functions.graphWorld import coordToint,intTocoord
from functions.features import get_feature_levels, get_feature_reward_sum

sim = pd.DataFrame(pd.read_pickle('data/tasksetup/exp1_simtrials.pkl'))
sim['OBM_advice_utility'] = [[0,0]]*len(sim)
sim['NBM_advice_utility'] = [[0,0]]*len(sim)
sim['POM_advice_utility'] = [[0,0]]*len(sim)
sim['PathAvgUtility'] = [[0,0]]*len(sim)
sim['Q-values'] = [[0,0]]*len(sim)
sim['feature_levels'] = [[0,0]]*len(sim)
sim['feature_reward_sum'] = [[0,0]]*len(sim)

for i,row in sim.iterrows():
    trial_param = create_true_mdp_params(0,4)
    trial_param['goal_values'] = row.learner_rewards
    obm_mentor = OptimalBayesianMentor(trial_param) # create Optimal Bayesian Mentor class
    OBM_advice_utility = obm_mentor.advice_dist([row.traj,]) # adivce utility based on OBM

    # trial_param = create_true_mdp_params(0,4)
    # trial_param['goal_values'] = row.learner_reward
    nbm_mentor = NaiveBayesianMentor(trial_param) 
    NBM_advice_utility = nbm_mentor.advice_dist([row.traj,]) 

    # trial_param = create_true_mdp_params(0,4)
    # trial_param['goal_values'] = row.learner_rewards
    pom_mentor = PriorOnlyMentor(trial_param)
    POM_advice_utility = pom_mentor.prior_advice_dist()
    
    PathAvgUtility = pathAvgUtility(trial_param['connections'],trial_param['goal_values'])
    
    mdp = create_mdp(trial_param)
    results = ValueIteration().plan_on(mdp=mdp)
    Q_value = q_values(results.Q, trial_param['connections'])
    

    feature_level = coordToint(get_feature_levels(trial_param['connections']))
    feature_reward = coordToint(get_feature_reward_sum(trial_param['goal_values'],trial_param['connections']))

    #save as sim[,OBM_advice_utility]
    sim.at[i, 'OBM_advice_utility'] = OBM_advice_utility
    sim.at[i, 'NBM_advice_utility'] = NBM_advice_utility
    sim.at[i, 'POM_advice_utility'] = POM_advice_utility
    sim.at[i,'feature_levels'] = intTocoord(feature_level)
    sim.at[i,'feature_reward_sum'] = intTocoord(feature_reward)
    sim.at[i,'PathAvgUtility'] = PathAvgUtility
    sim.at[i,'Q-values'] = Q_value


pd.to_pickle(sim, 'data/sim/exp1/exp1_modelsim.pickle')

In [1]:
import pandas as pd
import numpy as np
from functions.fitting import fitting_choices

preprocessed = pd.read_pickle('data/preprocessed/exp1/preprocessed_exp1.pkl')

models = {
    "PathAvgUtility": ["PathAvgUtility"],
    "Q-values": ["Q-values"],
    "Level": ["feature_levels"],
    "Reward": ["feature_reward_sum"],
    "Level,Reward": ["feature_levels", "feature_reward_sum"],
    "OBM": ["OBM_AU"],
    "NBM": ["NBM_AU"],
    "POM": ["POM_AU"],
}

df_fits_exp1 = fitting_choices(preprocessed, models,exp = 1)
df_fits_exp1.to_pickle('data/preprocessed/exp1/df_fits_exp1.pkl')

100%|██████████| 100/100 [05:00<00:00,  3.01s/it]


In [ ]:
import pandas as pd
import numpy as np
from functions.model_comparison import sample_multinomial_logit
from collections import defaultdict
from functions.functions import max_value_keys
            
simdata = pd.DataFrame(pd.read_pickle('data/sim/exp1_modelsim.pickle'))
rename_simdata_map = {
    "feature_levels": "Level",
    "feature_reward_sum": "Reward",
    "OBM_advice_utility": "OBM",
    "NBM_advice_utility": "NBM",
    "POM_advice_utility": "POM",
}
simdata.rename(columns=rename_simdata_map, inplace=True)

df_fits = pd.read_pickle('data/preprocessed/exp1/df_fits_exp1.pkl')
rename_df_fits_map = {
    "PathAvgUtility": "PathAvgUtility",
    "Q-values": "Q-values",
    "feature_levels": "Level",
    "feature_reward_sum": "Reward",
    "OBM_AU": "OBM",
    "NBM_AU": "NBM",
    "POM_AU": "POM",
}
df_fits.rename(columns=rename_df_fits_map,inplace=True)

models = {
    "PathAvgUtility": ["PathAvgUtility"],
    "Q-values": ["Q-values"],
    "Level": ["Level"],
    "Reward": ["Reward"],
    "Level,Reward": ["Level", "Reward"],
    "OBM": ["OBM"],
    "NBM": ["NBM"],
    "POM": ["POM"],
}

posterior_sim = pd.DataFrame({"samples": [],
                              "subjID": [],
                              "model": [],
                              "feature_weights": [],
                              "choice": [],
                              "teaching_score" : [],
                            })
simrows = []

Subjects = df_fits.subjID.unique()
K = df_fits.model.unique()
iteration = 100

for s in range(0,iteration):
    for subj in Subjects:
        subj_fits =  df_fits[df_fits.subjID == subj].reset_index(drop=True)
        for k in K:
            feature_weights = subj_fits[subj_fits.model == k][models[k]].values.flatten()
            for i, row in simdata.iterrows():
                feature_value_dict = row[models[k]]
                choice = sample_multinomial_logit(feature_weights,feature_value_dict)
                teaching_score = row.OBM[choice]/ row.OBM[max_value_keys(row.OBM)[0]] 
            
                simrow= {
                    "iteration": s,
                    "subjID": subj,
                    "sim_model": k,
                    'trial_id' : row.seed,
                    "feature_weights": feature_weights.tolist(),  
                    "choice": choice,
                    "teaching_score": teaching_score,
                }
                for model_name in models.keys():
                    if model_name != "Level,Reward":
                        simrow[model_name] = row[model_name] 

                simrows.append(simrow)

posterior_sim = pd.DataFrame.from_records(simrows)

# Split into two halves to reduce file size for GitHub (files > 100MB not allowed)
posterior_sim_0to50 = posterior_sim[posterior_sim['iteration'] < 50]
posterior_sim_50to100 = posterior_sim[posterior_sim['iteration'] >= 50]

# Save the two halves
posterior_sim_0to50.to_pickle('data/sim/exp1/exp1_posterior_sim_0to50.pkl')
posterior_sim_50to100.to_pickle('data/sim/exp1/exp1_posterior_sim_50to100.pkl')

In [ ]:
import importlib, functions.fitting
importlib.reload(functions.fitting)
from functions.fitting import model_recovery_fitting

sim_fits = model_recovery_fitting(posterior_sim, models)

sim_fits.to_pickle('data/sim/exp1/exp1_simfits_100.pkl')